In [2]:
import numpy as np
import random as rand

init_pop = 100
gene_num = 3
bacteria = []

mut_chance = 0.1
num_mut_places = 1

cross_chance = 0.8
num_cross_places = 1

selection_surv_percent = 0.05

epochs = 1000

def dectobin(n):
    return bin(n).replace("0b", "")

class Bacterium:

    def __init__(self, genotype, fenotype):
        self.genotype = genotype
        self.fenotype = fenotype

    def __str__(self):
        return "Bacterium G(" + str(self.genotype) + "), F(" + str(self.fenotype) + ")"

    def __repr__(self):
        return "Bacterium G(" + str(self.genotype) + "), F(" + str(self.fenotype) + ")"

    def __add__(self, other):
        splittedself = []
        splittedother = []
        splitplaces = []
        genotypeA = []
        genotypeB = []
        genotypeforA = []
        genotypeforB = []

        for i in range(len(self.genotype)):
            for j in self.genotype[i]:
                splittedself.append(j)
            for j in other.genotype[i]:
                splittedother.append(j)
        for i in range(num_cross_places):
            splitplaces.append(rand.randint(1,len(splittedself)-2))
        splitplaces.sort()

        for i in range(num_cross_places):
            genotypeA = splittedself[:splitplaces[i]] + splittedother[splitplaces[i]:]
            genotypeB = splittedother[:splitplaces[i]] + splittedself[splitplaces[i]:]

        for i in range(gene_num):
            genotypeforA.append(''.join(genotypeA[(i * 8):((i + 1) * 8)]))
            genotypeforB.append(''.join(genotypeB[(i * 8):((i + 1) * 8)]))

        # first.fenotype = 0
        # second.fenotype = 0
        # for i in range(len(first.genotype)):
        #     first.fenotype += int(first.genotype[i], 2)
        #     second.fenotype += int(second.genotype[i], 2)

        return Bacterium(genotypeforA, 0), Bacterium(genotypeforB, 0)

    def express(self):
        self.fenotype = 0
        for i in range(len(self.genotype)):
            self.fenotype += int(self.genotype[i], 2)
        return self


    def mutate(self):
        mutationlist = []
        weights = []
        splitted = []
        mutated = []
        for i in range(num_mut_places + 1):
            mutationlist.append(i)
        for i in mutationlist:
            if i == 0:
                weights.append(1 - mut_chance)
            else:
                weights.append(mut_chance / (len(mutationlist) - 1))

        howmanymuts = rand.choices(mutationlist, weights=weights)

        howmanymuts = howmanymuts[0]

        for i in range(len(self.genotype)):
            for j in self.genotype[i]:
                splitted.append(j)

        while howmanymuts != 0:
            pointmut = rand.randint(0, len(splitted)-1)

            if pointmut not in mutated:
                if splitted[pointmut] == '0':
                    splitted[pointmut] = '1'
                    howmanymuts = howmanymuts - 1
                else:
                    splitted[pointmut] = '0'
                    howmanymuts = howmanymuts - 1
                mutated.append(pointmut)
            else:
                continue

        for i in range(gene_num):
            self.genotype[i] = ''.join(splitted[(i*8):((i+1)*8)])


        self.fenotype = 0
        for i in range(len(self.genotype)):
            self.fenotype += int(self.genotype[i], 2)


def generatepopulation():
        for i in range(init_pop):
            genotype = []
            fenotype = 0
            for i in range(gene_num):
                genotype.append(dectobin(rand.randint(0,50)))
                while len(genotype[i]) != 8:
                    genotype[i] = '0' + genotype[i]
            for i in range(len(genotype)):
                fenotype += int(genotype[i],2)
            bacteria.append(Bacterium(genotype, fenotype))
        return bacteria

def pairing(bacteria):
    newpop = []
    for i in range(0, len(bacteria), 2):
        if rand.random() <= cross_chance:
            a = bacteria[i] + bacteria[i + 1]
            newpop.append(a[0])
            newpop.append(a[1])
        else:
            newpop.append(bacteria[i])
            newpop.append(bacteria[i+1])

    return newpop

def naturalselection(bacteria, ruletka=True):
    if ruletka:
        half = (len(bacteria)*selection_surv_percent)
        fenotypesum = 0
        weights = []
        newbacteria = []

        for i in bacteria:
            fenotypesum += i.fenotype
        for i in bacteria:
            try:
                weights.append((i.fenotype)/fenotypesum)
            except:
                weights = None

        while len(newbacteria) != half:
            winner = np.random.choice(bacteria, replace=False, p=weights)
            newbacteria.append(winner)

        while len(newbacteria) != len(bacteria):
            newbacteria.append(rand.choices(newbacteria))

        np.random.shuffle(newbacteria)

    else:
        best = []
        half = (len(bacteria)*selection_surv_percent)
        newbacteria = sorted(bacteria, key=lambda x: x.fenotype, reverse=True)

        newerbacteria = newbacteria[:int(half)]

        while len(best) != len(bacteria):
            best.append(np.random.choice(newerbacteria, replace=True))
        np.random.shuffle(best)
    return best


def checksum(bacterium):
    fenotype = 0
    splitted = []
    for i in range(len(bacterium.genotype)):
        splitted += [j for j in bacterium.genotype[i]]
    for i in range(len(bacterium.genotype)):
        fenotype += int(bacterium.genotype[i], 2)
    if fenotype == bacterium.fenotype:
        return True
    else:
        return False

bacteria = generatepopulation()

for i in range(epochs):

    # if i % 1 == 0:
    #     suma = 0
    #     std = []
    #     for j in bacteria:
    #         suma += j.fenotype
    #         std.append(j.fenotype)
    #
    #     print(i, '  ', suma/len(bacteria), ' ' , np.std(std), std)

    for j in bacteria:
        if rand.random() <= mut_chance:
            j.mutate()
        else:
            pass

    if i % 1 == 0:
        suma = 0
        std = []
        for j in bacteria:
            suma += j.fenotype
            std.append(j.fenotype)

        print(i, '  ', suma / len(bacteria), ' ', np.std(std), std)

    bacteria = pairing(bacteria)
    for j in bacteria:
        j.express()

    # if i % 1 == 0:
    #     suma = 0
    #     std = []
    #     for j in bacteria:
    #         suma += j.fenotype
    #         std.append(j.fenotype)
    # print(std)

    bacteria = naturalselection(bacteria, False)

    # if i % 1 == 0:
    #     suma = 0
    #     std = []
    #     for j in bacteria:
    #         suma += j.fenotype
    #         std.append(j.fenotype)
    # print(std)

0    81.17   27.06512700875427 [67, 93, 94, 101, 72, 126, 30, 115, 47, 61, 95, 76, 94, 68, 98, 65, 17, 122, 56, 95, 133, 98, 36, 66, 94, 140, 49, 70, 127, 125, 77, 96, 104, 69, 103, 46, 106, 91, 110, 63, 111, 112, 40, 73, 123, 60, 119, 39, 97, 45, 107, 50, 47, 79, 57, 58, 86, 91, 86, 102, 71, 80, 68, 64, 106, 118, 56, 62, 80, 58, 67, 107, 50, 79, 64, 106, 98, 89, 86, 121, 50, 87, 122, 69, 50, 124, 32, 85, 92, 51, 86, 87, 90, 100, 44, 31, 53, 74, 80, 103]
1    133.48   14.598273870564286 [123, 123, 128, 127, 127, 160, 128, 125, 123, 123, 127, 127, 123, 123, 128, 128, 127, 127, 125, 160, 127, 160, 123, 128, 125, 160, 123, 123, 125, 160, 160, 127, 125, 123, 160, 128, 127, 128, 128, 125, 160, 160, 125, 125, 128, 123, 127, 127, 123, 123, 160, 160, 160, 160, 125, 125, 125, 160, 125, 125, 160, 123, 123, 160, 123, 125, 127, 125, 125, 125, 123, 160, 160, 123, 128, 160, 160, 128, 128, 127, 128, 125, 123, 127, 125, 128, 160, 123, 160, 123, 128, 160, 127, 123, 128, 123, 128, 127, 128, 128]
2    19

In [5]:
import numpy as np
import random as rand

def f(x):
    return x[0]**2 + 2*x[0] + 1

def generate_genotype(n_features: int, feature_size=32) -> str:
    return ''.join([np.random.choice(['0', '1']) for _ in range(n_features * feature_size)])

# class Specimen:

random_geno = generate_genotype(2, 32)

In [6]:
random_geno

'1111110001010011110101010010100111010010000000000010001010101110'

In [124]:
from tqdm import tqdm
import numpy as np
import random as rand

class Specimen:
    def __init__(self, cost_function, cost_function_args = None, genotype: str = None, n_features: int = 2,
                 feature_size: int = 32, n_breed_points: int = 2, n_mutation_points: int = 2, mut_chance: float = 0.1,
                 float_divisions = 1):

        if genotype is None:
            self.genotype = generate_genotype(n_features, feature_size)
        else:
            self.genotype = genotype
        self.n_features = n_features
        self.cost_function = cost_function
        self.cost_function_args = cost_function_args if cost_function_args is not None else ()
        self.feature_size = feature_size
        self.n_breed_points = n_breed_points
        self.n_mutation_points = n_mutation_points
        self.mut_chance = mut_chance
        self.float_divisions = float_divisions
        self.score = None
        self.phenotype = None

    def calculate_score(self):
        if self.score is None:
            self.score = -self.cost_function(self.phenotype, *self.cost_function_args)
        else:
            return self.score
        return self.score

    def calculate_phenotype(self):
        phenotype = []
        for feature in range(self.n_features):
            phenotype.append(int(self.genotype[feature * self.feature_size:feature * self.feature_size + self.feature_size], 2) / self.float_divisions)
        self.phenotype = np.array(phenotype) #does not return anything !!!

    def breed(self, other):
        splitted_self = [bit for bit in self.genotype]
        splitted_other = [bit for bit in other.genotype]
        split_places = [rand.randint(1, len(splitted_self) - 2) for _ in range(self.n_breed_points)]
        split_places.sort()
        genotype_A = []
        genotype_B = []

        for idx in range(self.n_breed_points):
            genotype_A = splitted_self[:split_places[idx]] + splitted_other[split_places[idx]:]
            genotype_B = splitted_other[:split_places[idx]] + splitted_self[split_places[idx]:]

        merged_genotype_A = ''.join(genotype_A)
        merged_genotype_B = ''.join(genotype_B)

        return Specimen(cost_function=self.cost_function, cost_function_args=self.cost_function_args, genotype=merged_genotype_A,
                        n_features=self.n_features, feature_size=self.feature_size, n_breed_points=self.n_breed_points,
                        n_mutation_points=self.n_mutation_points, float_divisions=self.float_divisions,
                        mut_chance=self.mut_chance), \
               Specimen(cost_function=self.cost_function, cost_function_args=self.cost_function_args, genotype=merged_genotype_B,
                        n_features=self.n_features, feature_size=self.feature_size, n_breed_points=self.n_breed_points,
                        n_mutation_points=self.n_mutation_points, float_divisions=self.float_divisions,
                        mut_chance=self.mut_chance)

    def mutate(self):
        mutation_list = [idx for idx in range(self.n_mutation_points + 1)]
        weights = []
        splitted = [bit for bit in self.genotype]
        mutated = []

        for i in mutation_list:
            if i == 0:
                weights.append(1 - self.mut_chance)
            else:
                weights.append(self.mut_chance / (len(mutation_list) - 1))

        mutations_left = rand.choices(mutation_list, weights=weights)[0]

        while mutations_left != 0:
            point_mutation = rand.randint(0, len(splitted)-1)

            if point_mutation not in mutated:
                if splitted[point_mutation] == '0':
                    splitted[point_mutation] = '1'
                    mutations_left = mutations_left - 1
                else:
                    splitted[point_mutation] = '0'
                    mutations_left = mutations_left - 1
                mutated.append(point_mutation)
            else:
                continue

        self.genotype = ''.join(splitted) #calculate fenotype after

    def __str__(self):
        return str(self.phenotype)

    def __repr__(self):
        return str(self.phenotype)


class GeneticAlgorithm:
    def __init__(self, cost_function, cost_function_args = None, n_specimens: int = 100, breed_chance: float = 0.8,
                 n_breed_points: int = 2, mut_chance: float = 0.1, n_mutation_points: int = 10, n_features: int = 2,
                 feature_size: int = 32, float_divisions: int = 1, n_jobs: int = 1):

        self.cost_function = cost_function
        self.cost_function_args = cost_function_args
        self.n_specimens = n_specimens
        self.breed_chance = breed_chance
        self.n_breed_points = n_breed_points
        self.mut_chance = mut_chance
        self.n_mutation_points = n_mutation_points
        self.n_features = n_features
        self.feature_size = feature_size
        self.float_divisions = float_divisions
        self.n_jobs = n_jobs
        self.population = [Specimen(self.cost_function, n_features=self.n_features,
                                            feature_size=self.feature_size,
                                            n_breed_points=self.n_breed_points,
                                            n_mutation_points=self.n_mutation_points,
                                            mut_chance=self.mut_chance,
                                            float_divisions=self.float_divisions,
                                            cost_function_args=self.cost_function_args) for _ in range(self.n_specimens)]

    def natural_selection(self, roulette=True):
        for specimen in self.population:
            specimen.calculate_phenotype()
            specimen.calculate_score()

        if roulette:
            n_survivors = (len(self.population) * self.breed_chance)
            fenotype_sum = 0
            weights = []
            new_population = []

            for specimen in self.population:
                fenotype_sum += specimen.score

            for specimen in self.population:
                try:
                    weights.append(specimen.score / fenotype_sum)
                except:
                    weights = None

            while len(new_population) != n_survivors:
                winner = np.random.choice(self.population, replace=False, p=weights)
                new_population.append(winner)

            while len(new_population) != len(self.population):
                new_population.append(rand.choices(new_population))

            np.random.shuffle(new_population)
            self.population = new_population

        else:
            new_population = []
            n_survivors = (len(self.population) * self.breed_chance)
            sorted_population = sorted(self.population, key=lambda x: x.score, reverse=True)

            fittest = sorted_population[:int(n_survivors)]

            while len(new_population) != len(self.population):
                new_population.append(np.random.choice(fittest, replace=True))

            np.random.shuffle(new_population)
            self.population = new_population

    def breed_population(self):
        new_population = []
        for idx in range(0, len(self.population), 2):
            specimen_A = self.population[idx]
            specimen_B = self.population[idx + 1]

            if rand.random() <= self.breed_chance:
                child_A, child_B = specimen_A.breed(specimen_B)
                new_population.append(child_A)
                new_population.append(child_B)
            else:
                new_population.append(specimen_A)
                new_population.append(specimen_B)
        self.population = new_population

    def run(self, n_iter:int = 100, verbose: bool = True):
        title = f""
        pbar = tqdm(range(n_iter), disable=not verbose, desc=title)
        for _ in pbar:

            self.breed_population()

            for specimen in self.population:
                specimen.mutate()

            self.natural_selection(roulette=False)

            for specimen in self.population:
                specimen.calculate_phenotype()
                specimen.calculate_score()

            scores = np.array([specimen.score for specimen in self.population])
            best_score = max(scores)
            title = f"Current best score: {-best_score:.4f}"
            pbar.set_description(title)

    def get_best(self):
        scores = np.array([specimen.score for specimen in self.population])
        best_score = max(scores)
        best_phenotype = self.population[np.argmax(scores)].phenotype
        return best_phenotype, -best_score

In [125]:
engine = GeneticAlgorithm(cost_function=f, n_specimens=100, n_features=2, feature_size=32,
                       n_breed_points=3, n_mutation_points=2, float_divisions=100, n_jobs=6,
                       mut_chance=0.1, breed_chance=0.5)

engine.run(n_iter=100)

Current best score: 1.0000: 100%|██████████| 100/100 [00:01<00:00, 89.82it/s]           


In [116]:
szymke = Specimen(cost_function=f)
szymke.genotype
szymke.calculate_phenotype()
szymke.calculate_score()

-1.2287801046708079e+19

In [106]:
szymke.mut_chance = 0.99
szymke.n_mutation_points = 20
szymke.mutate()
second = szymke.genotype